In [1]:
import polars as pl
import pysentiment2 as ps
import math
import os
import re
import urllib.parse

In [2]:
lm_dict = pl.read_csv(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\Loughran-McDonald_MasterDictionary_1993-2021.csv')
hv_dict = pl.read_excel(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\converter_api\Harvard_inquirerbasic.xls')
hv_dict

Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,Weak,Submit,Active,Passive,Pleasur,Pain,Feel,Arousal,EMOT,Virtue,Vice,Ovrst,Undrst,Academ,Doctrin,Econ@,Exch,ECON,Exprsv,Legal,Milit,Polit@,POLIT,Relig,Role,COLL,Work,…,WltTran,WltOth,WltTot,WlbGain,WlbLoss,WlbPhys,WlbPsyc,WlbPt,WlbTot,EnlGain,EnlLoss,EnlEnds,EnlPt,EnlOth,EnlTot,SklAsth,SklPt,SklOth,SklTot,TrnGain,TrnLoss,TranLw,MeansLw,EndsLw,ArenaLw,PtLw,Nation,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,null,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""A""","""H4Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""DET ART""","""| article: Ind…"
"""ABANDON""","""H4Lvd""",null,"""Negativ""",null,null,"""Ngtv""",null,null,null,"""Weak""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""SUPV""","""|"""
"""ABANDONMENT""","""H4""",null,"""Negativ""",null,null,null,null,null,null,"""Weak""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Noun""","""|"""
"""ABATE""","""H4Lvd""",null,"""Negativ""",null,null,null,null,null,null,null,null,null,"""Passive""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""TranLw""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""SUPV""","""|"""
"""ABATEMENT""","""Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Noun""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZENITH""","""H4""","""Positiv""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Noun""","""|"""
"""ZERO""","""H4Lvd""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""DET""","""|"""
"""ZEST""","""H4""","""Positiv""",null,null,null,null,null,"""Strong""",null,null,null,null,null,null,null,null,"""Arousal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,nu

In [3]:
#ensure that harvard dictionary entries are compatable with input strings
words = hv_dict.select(pl.col('Entry'))

hv_words = words.to_series().str.to_lowercase()

hv_words = hv_words.map_elements(lambda word: re.sub(r'#\d+', '', word))

hv_dict = hv_dict.replace_column(0, hv_words)

hv_dict = hv_dict.unique(subset="Entry", keep="last")


In [4]:
df_dict = {}

path = r'C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd'

for company_name in os.listdir(path):
    
    file_path = os.path.join(path, company_name)

    match = re.search(r"\\([A-Z]+(?:\.[A-Z])?)_reports\.parquet", file_path)

    if match:
        ticker_symbol = match.group(1)
        print("Ticker Symbol:", ticker_symbol, '\n', file_path)
    else:
        print(f"Ticker symbol for {company_name} is not found")

    df_name = f"{ticker_symbol}"
    df = pl.read_parquet(file_path)

    df_dict[df_name] = df

Ticker Symbol: AAL 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\AAL_reports.parquet
Ticker Symbol: AAPL 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\AAPL_reports.parquet
Ticker Symbol: ABBV 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\ABBV_reports.parquet
Ticker Symbol: ABNB 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\ABNB_reports.parquet
Ticker Symbol: ABT 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\ABT_reports.parquet
Ticker Symbol: ACGL 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\ACGL_reports.parquet
Ticker Symbol: ACN 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\ACN_reports.parquet
Ticker Symbol: ADBE 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred_upd\ADBE_reports.parquet
Ticker Symbol: ADI 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_f

In [5]:
df_dict['AAL']

2019-02-25_66814,2019-04-26_27720,2019-07-25_30317,2019-10-24_31343,2020-02-19_63134,2020-04-30_38929,2020-07-23_44821,2020-10-22_56671,2021-02-17_98273,2021-04-22_48588,2021-07-22_52761,2021-10-21_54201,2022-02-22_103357,2022-04-21_41461,2022-07-21_44034,2022-10-20_44610,2023-02-22_84935,2023-04-27_39746,2023-07-20_41797,2023-10-19_41931
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""delinquent""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""best""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""knowledge""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""definitive""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,null,null,null,null,"""substitution""",null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,"""authority""",null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,"""virtue""",null,null,null,null,null,null,null


In [5]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

9830


In [6]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

21144808


In [7]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
print(f'Average doc length in words: {round(average_word_count)}')
ln_average_word_count = math.log(average_word_count)

Average doc length in words: 2151


In [122]:
#ENSURE CASE CoMpAtAbIlItY
positive_words_lm = lm_dict.filter(lm_dict["Positive"] > 0).to_series().str.to_lowercase()
negative_words_lm = lm_dict.filter(lm_dict["Negative"] > 0).to_series().str.to_lowercase()

print(len(positive_words_lm) + len(negative_words_lm))

positive_words_hv = hv_dict.filter(hv_dict["Positiv"] == 'Positiv').to_series().str.to_lowercase()
negative_words_hv = hv_dict.filter(hv_dict["Negativ"] == 'Negativ').to_series().str.to_lowercase()

print(len(positive_words_hv) + len(negative_words_hv))


2692
3475


In [9]:
#example series 
example_series = df_dict['ALGN'][df_dict['ALGN'].columns[1]]
example_df = df_dict['ALGN']
example_df


2019-02-28_41967,2019-05-02_23700,2019-08-01_25159,2019-10-31_25727,2020-02-28_41700,2020-05-05_29591,2020-07-31_31169,2020-10-30_31283,2021-02-26_51814,2021-05-05_24843,2021-08-04_27369,2021-11-02_29192,2022-02-25_51116,2022-05-05_31211,2022-08-04_31365,2022-11-04_32365,2023-02-27_48299,2023-05-05_25156,2023-08-04_26049,2023-11-03_26861
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal""","""principal"""
"""delinquent""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""best""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""knowledge""","""company""","""company""","""company""","""shell""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
"""definitive""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company""","""company"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,null,null,null,null,"""omitted""",null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,"""behalf""",null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,null,null,null,null,"""substitution""",null,null,null,null,null,null,null


This is the func, that appled to a series and get sentiment words from it. As an output we get a df with first col - grouped words that appeaps in LOUGHRAN dictionary and has >0 value in "positive" or "negative" cols. Secon col contain count of corresponding word in this document. Third col contain boolean, indicatin if this word has positive sentiment (True) or negative (False)

In [121]:
def count_sentiment_w_in_doc(series):
        
        #grouped series that became a df, where each row is unique word and its count in document 
        num_of_w = series.value_counts().rename({"count": "count in doc"})

        #print(num_of_w.shape)
        
        #first col of this df (only words by itself)
        only_w = num_of_w[num_of_w.columns[0]]
        

        #Filterd df, there is only words that have positive or negative sentiment score in this document 
        pos_w_lm = num_of_w.filter(only_w.is_in(positive_words_lm)).with_columns(pl.lit(True).alias("positive_lm"))
        neg_w_lm = num_of_w.filter(only_w.is_in(negative_words_lm)).with_columns(pl.lit(False).alias("positive_lm"))
        #print(pos_w_lm.shape[0], neg_w_lm.shape[0])

        pos_w_hv = num_of_w.filter(only_w.is_in(positive_words_hv)).with_columns(pl.lit(True).alias("positive_hv"))
        neg_w_hv = num_of_w.filter(only_w.is_in(negative_words_hv)).with_columns(pl.lit(False).alias("positive_hv"))
        #print(pos_w_hv.shape[0], neg_w_hv.shape[0])

        #Staked for one df for further operations 
        df_stacked_lm = neg_w_lm.vstack(pos_w_lm)
        df_stacked_hv = neg_w_hv.vstack(pos_w_hv)
        #print(f'df_stacked_lm: {df_stacked_lm.shape}; df_stacked_hv: {df_stacked_hv}')

        total_df = num_of_w.join(df_stacked_lm, on=series.name, how='left', validate='1:1')
        total_df = total_df.drop('count in doc_right')
        #print(total_df.shape)

        total_df = total_df.join(df_stacked_hv, on=series.name, how='left', validate='1:1')
        total_df = total_df.drop('count in doc_right')
        #print(total_df)
  

        #print(df_stacked_lm.shape, df_stacked_hv.shape)

        #total_df = df_stacked_lm.join(df_stacked_hv, on=df_series.name, how='outer')
        #print(total_df)

        return total_df

check = count_sentiment_w_in_doc(example_series)  
print(check)

shape: (428, 4)
┌──────────────────┬──────────────┬─────────────┬─────────────┐
│ 2019-05-02_23700 ┆ count in doc ┆ positive_lm ┆ positive_hv │
│ ---              ┆ ---          ┆ ---         ┆ ---         │
│ str              ┆ u32          ┆ bool        ┆ bool        │
╞══════════════════╪══════════════╪═════════════╪═════════════╡
│ differ           ┆ 7            ┆ null        ┆ false       │
│ pay              ┆ 2            ┆ null        ┆ null        │
│ correct          ┆ 2            ┆ null        ┆ null        │
│ terminate        ┆ 1            ┆ false       ┆ null        │
│ disgorgement     ┆ 2            ┆ false       ┆ null        │
│ …                ┆ …            ┆ …           ┆ …           │
│ duty             ┆ 2            ┆ null        ┆ false       │
│ profitability    ┆ 7            ┆ true        ┆ null        │
│ criminal         ┆ 1            ┆ false       ┆ false       │
│ disrupt          ┆ 6            ┆ false       ┆ false       │
│ charge           ┆ 3  

This is the func that counts a number of docs with at least one occurense of the input word. It takes hole dictionary whith dataframes and particular word as an input. Then checks how many cols in each df contain this word, sum up all resulst and return a single value for each word.

In [79]:
def in_all_doc_occurence(check_str, df_dict):
    count = 0
    for df in df_dict.values():
        count += df.select((pl.all() == check_str).sum() > 0).sum_horizontal()[0]
        
    return count

In [80]:
stringg = "a"
in_all_doc_occurence(stringg, df_dict)

0

This is the func that takes a df from count_sentiment_w_in_doc() and for each word in this df applies in_all_doc_occurence(), meaning that it counts all docs with at least one occurence of this word. The result of applying this func for each word in df is a series of integers showing counts for correcdonding word. This series augmented af col named "count in all docs"

In [81]:
def total_occurence_count(df_series, df_dict):
   check = count_sentiment_w_in_doc(df_series).lazy()

   counts = check.with_columns(pl.col(check.columns[0]).map_elements(lambda x: in_all_doc_occurence(x, df_dict)).alias("count in all docs")).collect()
   return counts

In [104]:
res_df = total_occurence_count(example_series, df_dict)
print(res_df)

shape: (428, 5)
┌──────────────────┬──────────────┬─────────────┬─────────────┬───────────────────┐
│ 2019-05-02_23700 ┆ count in doc ┆ positive_lm ┆ positive_hv ┆ count in all docs │
│ ---              ┆ ---          ┆ ---         ┆ ---         ┆ ---               │
│ str              ┆ u32          ┆ bool        ┆ bool        ┆ i64               │
╞══════════════════╪══════════════╪═════════════╪═════════════╪═══════════════════╡
│ transparency     ┆ 1            ┆ true        ┆ null        ┆ 3061              │
│ offset           ┆ 9            ┆ null        ┆ true        ┆ 9777              │
│ profitability    ┆ 7            ┆ true        ┆ null        ┆ 6470              │
│ favor            ┆ 1            ┆ null        ┆ true        ┆ 2958              │
│ lose             ┆ 7            ┆ false       ┆ null        ┆ 3121              │
│ …                ┆ …            ┆ …           ┆ …           ┆ …                 │
│ authoritative    ┆ 1            ┆ null        ┆ true      

This is the func that takes as an argument resulting df, wich was obtained from total_occurence_count(). Recal that this df contains information about one document (one col in df). This func calculate weighted and raw scores using precomputed metrics. Returns a list with weighted score and raw score. Weights are calculated based on this formula: 

$$
w_{i,j} = 
\begin{cases} 
\left(\frac{1 + \log(tf_{i,j})}{1 + \log(a)}\right) \log\left(\frac{N}{df_i}\right) & \text{if } tf_{i,j} \geq 1 \\
0 & \text{otherwise} 
\end{cases}
$$


N represents the total number of 10-Ks in the sample, dfi the number of documents containing at least one occurrence of the i th word, tfi,j the raw count of the i th word in the j th document, and a the average word count in the document.

The formula is from:

Loughran T., McDonald B. When is a liability not a liability? Textual analysis, dictionaries, and 10‐Ks //The Journal of finance. – 2011. – Т. 66. – №. 1. – С. 35-65.

In [119]:
def compute_score(res_df):

    res_df.lazy()

    res = res_df.with_columns((((1 + pl.col(res_df.columns[1]).log())/(1 + ln_average_word_count))*((total_num_of_rep/pl.col(res_df.columns[4])).log())).alias("metrics"))

    
    lm_positive = res.filter(pl.col(('positive_lm')) == True)
    w_pos_sen_lm = lm_positive.select(pl.col(res.columns[5])).sum().item()

    lm_negative = res.filter(pl.col(('positive_lm')) == False)
    w_neg_sen_lm = lm_negative.select(pl.col(res.columns[5])).sum().item()

    hv_positive = res.filter(pl.col(('positive_hv')) == True)
    w_pos_sen_hv = hv_positive.select(pl.col(res.columns[5])).sum().item()

    hv_negative = res.filter(pl.col(('positive_hv')) == False)
    w_neg_sen_hv = hv_negative.select(pl.col(res.columns[5])).sum().item()

    w_score_lm = w_pos_sen_lm - w_neg_sen_lm
    w_score_hv = w_pos_sen_hv - w_neg_sen_hv

    #lm_score = lm_positive.select(pl.col("count in doc")).sum().item() - lm_negative.select(pl.col("count in doc")).sum().item()
    #hv_score = hv_positive.select(pl.col("count in doc")).sum().item() - lm_negative.select(pl.col("count in doc")).sum().item()
    #print(lm_score, hv_score)

    #w_pos_sen = res.filter(pl.col('positive_lm') == True).select(pl.col(res.columns[5])).sum().item()
    #w_neg_sen = res.filter(pl.col('positive_lm') == False).select(pl.col(res.columns[5])).sum().item()
#
    #w_score = w_pos_sen - w_neg_sen
    #score = res.select(pl.col('positive')).sum().item()*2 - len(res)
#
    scores = [w_score_lm, w_score_hv]
    
    return scores

compute_score(res_df)

[-43.34217407016732, -3.8463984261083795]

This is the function that takes polars series as an input, then concatinate all values in the single string. For this sitring, which is the initial report, this func calculates sentiment scores according to Harvard IV-4 and Loughran and McDonald Financial Sentiment Dictionaries. Perticulary, this function calculates polarity score which is defined by the following formula:

$$
Polarity = \frac{N_{\text{pos}} - N_{\text{neg}}}{N_{\text{pos}} + N_{\text{neg}}}
$$

In [124]:
hiv4 = ps.HIV4()
lm = ps.LM() 

def compute_pysentiment(series):

    doc_string = series.str.concat(" ", ignore_nulls=True)

    tokens_hiv4 = hiv4.tokenize(doc_string[0])
    tokens_lm = lm.tokenize(doc_string[0])

    score_hiv4 = hiv4.get_score(tokens_hiv4)
    score_lm = lm.get_score(tokens_lm)

  
    return score_lm["Polarity"], score_hiv4["Polarity"]

In [125]:
#Example of pysentiment2 working on example_series: first value is "Polsrity" score from Loughran-McDonald dictionary, and second one is from Harvard dictionary
lst = [1, 48.0303, 334, 'extended values starts from here:']

lst.extend(compute_pysentiment(example_series))

print(lst)

[1, 48.0303, 334, 'extended values starts from here:', -0.6083150978027188, 0.11774842038038573]


In [118]:
pattern = r"_(\d+)$"

def get_doc_len(df):
    string_with_len = df.columns[0]

    match = re.search(pattern, string_with_len)
    doc_len = match.group(1)

    return doc_len


General loop that iterates over dfs and over it columns. It uses predefined finctions to calculate scores and save it as parquet files. Each parquet file named with company name is a dataframe with col names as field dates, first row is weighted scores and second row is raw scores. 

In [127]:
for company_name, df in df_dict.items():
    counter = 0

    score_dict = {}

    for col in df:
        
        res_df = total_occurence_count(col, df_dict)
        print('res df is ready')
        doc_len = get_doc_len(res_df)
        scores = compute_score(res_df)

        scores.extend(compute_pysentiment(col))
        scores.append(doc_len)

        score_dict[col.name] = scores

    counter += 1
    
    df = pl.DataFrame(score_dict)
    
    print(f'Iteration for {company_name} is done successfully; \n {len(df_dict) - counter} remaining')

 # Determine the output directory and file name
    output_dir = os.path.join('data', 'snp_500_scores_upd')
    os.makedirs(output_dir, exist_ok=True)

    file_name_new = f"{company_name}_full_doc_sen_score_df.parquet"
    full_path = os.path.join(output_dir, file_name_new)
    full_path = os.path.normpath(full_path)

    print(f"Attempting to write to: {full_path}")

    # Write the DataFrame to Parquet
    df.write_parquet(full_path)

    print(f"{full_path} created successfully")
        
        

res df is ready
res df is ready


PanicException: python function failed KeyboardInterrupt: 

PanicException: python function failed KeyboardInterrupt: 